## 单进程指定GPU进行计算

在Pytorch框架下，单进程指定GPU进行计算核心语句：`device = torch.device(“cuda:gpu编号”)`
例如： `device = torch.device(“cuda:1”)`

GPU编号、显存大小、当前正在使用GPU的进程都可通过命令行语句 `nvidia-smi` 查看

此外，还可以通过 `CUDA_VISIBLE_DEVICES` 变量指定GPU

对于模型（类），使用 `.to(device)` 语句来将模型的所有参数、缓存放到指定的GPU上进行计算

注：只要显存足够，多个terminal分别执行多个进程，可以指定在同一个GPU，也可以指定在不同的GPU，即可实现多进程并行

In [ ]:
'''
Demo, 单进程指定GPU计算
'''

import torch

# 1. 检查可用的GPU数量和编号
device_count = torch.cuda.device_count()
print(f"系统中有 {device_count} 块可用的GPU。")
if device_count > 0:
    print("它们的编号是：")
    for i in range(device_count):
        print(f"GPU {i}")
else:
    print("没有可用的GPU。")

# 2. 随机生成两个可乘矩阵，并在指定的GPU上进行乘法运算
if device_count > 0:
    # 让用户输入想要使用的GPU编号
    gpu_id = int(input("请输入你想要使用的GPU编号(0到{}):".format(device_count - 1)))
    
    # 检查输入的GPU编号是否有效
    if gpu_id < 0 or gpu_id >= device_count:
        print("输入的GPU编号无效。")
    else:
        # 指定设备
        device = torch.device(f"cuda:{gpu_id}")
        
        # 在指定的GPU上创建两个随机的可乘矩阵
        matrix1 = torch.rand(3, 3, device=device)
        matrix2 = torch.rand(3, 4, device=device)
        
        print(f"在GPU {gpu_id} 上的随机矩阵1:\n{matrix1}")
        print(f"在GPU {gpu_id} 上的随机矩阵2:\n{matrix2}")
        
        # 执行矩阵乘法
        result = torch.matmul(matrix1, matrix2)
        
        print(f"矩阵乘法的结果:\n{result}")
else:
    print("由于没有可用的GPU，无法执行GPU上的矩阵乘法。")


In [ ]:
'''
Demo, 指定进程的GPU可见范围

CUDA_VISIBLE_DEVICES="1"           Only device 1 will be seen
CUDA_VISIBLE_DEVICES="0,1"         Devices 0 and 1 will be visible
CUDA_VISIBLE_DEVICES="0,1"         Same as above, quotation marks are optional
CUDA_VISIBLE_DEVICES="0,2,3"       Devices 0, 2, 3 will be visible; device 1 is masked
CUDA_VISIBLE_DEVICES=""            No GPU will be visible
'''

import os
import torch

# 设置环境变量，不使用GPU
os.environ["CUDA_VISIBLE_DEVICES"] = ""

# 指定设备为GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print("当前使用的设备:", device)

# 在指定的GPU上创建两个可以做乘法的矩阵
matrix1 = torch.rand(2, 3, device=device)
matrix2 = torch.rand(3, 2, device=device)

# 打印生成的矩阵
print("Matrix 1:")
print(matrix1)
print("Matrix 2:")
print(matrix2)

# 进行矩阵乘法
result = torch.matmul(matrix1, matrix2)

# 打印结果
print("Result of matrix multiplication:")
print(result)

In [ ]:
'''
Demo, 把模型放到GPU上做计算
'''

import torch
import torch.nn as nn
import torch.nn.functional as F

# 定义一个简单的神经网络模型
class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.fc = nn.Linear(10, 5) 
        # self.to(torch.device("cuda:1")) # 定义网络时就指定设备

    # 前向传播过程
    def forward(self, x):
        x = self.fc(x)
        return F.relu(x) 

# 指定GPU设备
device = torch.device("cuda:1")

# 实例化模型
model = SimpleNet()

# 将模型移动到指定的设备
model.to(device)

# 创建一个随机数据张量，模拟输入数据
input_data = torch.randn(5, 10) 

# 将输入数据也移动到指定的设备，数据与模型在同一设备上可以提高计算效率
input_data = input_data.to(device)

# 前向传播，获取模型输出
output = model(input_data)

# 打印输出结果
print(output)

## 单进程跨GPU做计算

在深度学习和分布式计算领域，DP 通常指的是 DataParallel。DataParallel 是一种将计算任务在多个 GPU 上并行执行的方法。它在单机多卡环境中非常有用，可以在多个 GPU 上分摊工作负载，从而加快训练速度。

torch.nn.DataParallel 是 PyTorch 中的一个工具，可以让模型在多个 GPU 上并行运行。它通过将输入批次拆分成多个子批次，每个子批次发送到不同的 GPU 上，并行执行前向传播和反向传播，然后将每个 GPU 上的梯度聚合到主 GPU 上进行参数更新。

使用 DataParallel 的基本步骤
- 定义模型: 创建神经网络模型
- 包装模型: 使用 torch.nn.DataParallel 包装模型
- 将模型和数据迁移到 GPU: 使用 .to(device) 将模型和输入数据迁移到合适的设备上
- 训练模型: 按照常规方式训练模型

单机多卡训练策略
- 数据拆分，模型不拆分
- 数据不拆分，模型拆分
- 数据拆分，模型拆分

DataParallel 的局限性
- 数据并行粒度: DataParallel 进行的是数据并行操作，每个 GPU 处理一部分数据批次。由于其自动分配负载，这可能导致 GPU 利用率不均衡，尤其是在有计算负载差异的情况下
- 单节点限制: DataParallel 主要用于单节点多 GPU，即单机多卡。如果需要跨节点并行（分布式训练，多机多卡），应该考虑使用 torch.nn.parallel.DistributedDataParallel

In [ ]:
'''
Demo, 单机多卡, 单进程跨GPU计算

最简单高效的策略: 数据拆分, 模型不拆分
'''

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

# 设置环境变量，指定使用的GPU
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"

# 定义设备
globalDevice = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 定义一个简单的CNN模型
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv = nn.Conv2d(3, 16, 3, 1)
        self.fc = nn.Linear(16 * 26 * 26, 10)

    def forward(self, x):
        x = self.conv(x)
        x = torch.relu(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

# 实例化模型
cnn = CNN().to(globalDevice)

# 检查GPU数量并设置DataParallel
if torch.cuda.device_count() > 1:
    print(f"Using {torch.cuda.device_count()} GPUs")
    net = nn.DataParallel(cnn)
else:
    print("Using single GPU or CPU")
    net = cnn

# 定义数据集和数据加载器
class SimpleDataset(Dataset):
    def __init__(self, size):
        self.size = size

    def __len__(self):
        return self.size

    def __getitem__(self, idx):
        return torch.randn(3, 28, 28), torch.tensor(1)

dataset = SimpleDataset(1000)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True, num_workers=20)

# 定义优化器和损失函数
optimizer = optim.SGD(net.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

# 简单的训练过程
for epoch in range(5):
    for inputs, labels in dataloader:
        inputs, labels = inputs.to(globalDevice), labels.to(globalDevice) # 此处可指定GPU以实现手动分配负载
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1}, Loss: {loss.item()}")


In [ ]:
'''
Pseudocode，策略一：数据拆分，模型不拆分

在这种策略中，将数据拆分成多个批次，每个批次在一个GPU上进行处理。模型不会拆分，而是复制到每个GPU上
'''

import torch  
import torch.nn as nn  
import torch.optim as optim  
from torch.utils.data import DataLoader, Dataset  
from torch.nn.parallel import DataParallel  

# 假设我们有一个自定义的数据集和模型  
class MyDataset(Dataset):  
    # 实现__len__和__getitem__方法  
    pass  
class MyModel(nn.Module):  
    # 定义模型结构  
    pass  

# 初始化数据集和模型  
dataset = MyDataset()  
dataloader = DataLoader(dataset, batch_size=32, shuffle=True, num_workers=4)  
model = MyModel()  

# 检查GPU数量  
device_ids = list(range(torch.cuda.device_count()))  
model = DataParallel(model, device_ids=device_ids).to(device_ids[0])  

# 定义损失函数和优化器  
criterion = nn.CrossEntropyLoss()  
optimizer = optim.Adam(model.parameters(), lr=0.001)  

# 训练循环  
for epoch in range(num_epochs):  
    for inputs, labels in dataloader:  
        inputs, labels = inputs.to(device_ids[0]), labels.to(device_ids[0])  
        optimizer.zero_grad()  
        outputs = model(inputs)  
        loss = criterion(outputs, labels)  
        loss.backward()  
        optimizer.step()

In [ ]:
'''
Pseudocode，策略二：数据不拆分，模型拆分

在这种策略中，整个数据集在每个GPU上都会有一份副本，但模型会被拆分成多个部分，每个部分在一个GPU上运行。这种策略通常不常见，因为数据复制会消耗大量内存，而且模型拆分也可能会导致通信开销增加
'''
# 假设我们有一个可以拆分的模型（例如，具有多个子网络的模型）  
class SplitModel(nn.Module):  
    def __init__(self):  
        super(SplitModel, self).__init__()  
        self.subnet1 = nn.Sequential(...)  # 定义子网络1  
        self.subnet2 = nn.Sequential(...)  # 定义子网络2  
        # ... 其他子网络 ...  
    def forward(self, x):  
        # 前向传播逻辑，可能涉及跨多个设备的通信和数据传输  
        pass  

# 初始化模型和数据集（这里不实际拆分数据）  
model = SplitModel()  
dataset = MyDataset()  

# 将模型的每个子网络分配到一个GPU上  
model.subnet1 = model.subnet1.to('cuda:0')  
model.subnet2 = model.subnet2.to('cuda:1')  

# ... 其他子网络 ...  

# 训练循环（这里省略了数据加载和批处理，因为数据没有拆分）  
for epoch in range(num_epochs):  
    inputs, labels = ...  # 加载数据  
    inputs = inputs.to('cuda:0')  # 假设输入数据首先被送到第一个GPU上  
    optimizer.zero_grad()  
    outputs = model(inputs)  # 前向传播可能涉及跨多个GPU的通信  
    loss = criterion(outputs, labels)  
    loss.backward()  
    optimizer.step()


In [ ]:
'''
Pseudocode，策略三：数据拆分，模型拆分

在这种策略中，同时使用数据并行和模型并行。数据被拆分成多个批次，每个批次在不同的GPU上进行处理，同时模型也被拆分成多个部分，每个部分在不同的GPU上运行。这通常用于非常大的模型，单个GPU无法容纳整个模型的情况
'''

import torch  
import torch.distributed as dist  
import torch.nn as nn  
import torch.optim as optim  
from torch.utils.data import DataLoader, Dataset, DistributedSampler  
from torch.nn.parallel import DistributedDataParallel as DDP  

# 自定义数据集和模型  
class MyDataset(Dataset):  
    # 实现__len__和__getitem__方法  
    pass  
class MyModel(nn.Module):  
    # 定义模型结构，可能需要考虑如何拆分模型  
    pass  

# 初始化分布式环境  
dist.init_process_group(backend='nccl', init_method='tcp://localhost:23456', rank=0, world_size=torch.cuda.device_count())  

# 初始化数据集和模型  
dataset = MyDataset()  
sampler = DistributedSampler(dataset)  
dataloader = DataLoader(dataset, batch_size=32, shuffle=False, sampler=sampler)  
model = MyModel()  

#拆分模型（这通常需要根据模型的具体结构来手动完成。例如，如果模型有两个主要部分，可以将它们分别放到不同的设备上  
model_part1 = model.part1.to('cuda:0')  
model_part2 = model.part2.to('cuda:1')  

# 使用DistributedDataParallel包装模型  
model = DDP(model, device_ids=[torch.cuda.current_device()])

# 定义损失函数和优化器  
criterion = nn.CrossEntropyLoss()  
optimizer = optim.Adam(model.parameters(), lr=0.001)  

# 训练循环  
for epoch in range(num_epochs):  
    for inputs, labels in dataloader:  
        inputs, labels = inputs.to(model.device), labels.to(model.device)  
        optimizer.zero_grad()  
        outputs = model(inputs)  
        loss = criterion(outputs, labels)  
        loss.backward()  
        optimizer.step()  

# 销毁分布式进程组  
dist.destroy_process_group()

## 8个进程，全在单块4090显卡跑

耗时49.44012928009033秒

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import os
import multiprocessing
import time

# 定义一个简单的CNN模型
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv = nn.Conv2d(3, 16, 3, 1)
        self.fc = nn.Linear(16 * 26 * 26, 10)

    def forward(self, x):
        x = self.conv(x)
        x = torch.relu(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

# 定义数据集和数据加载器
class SimpleDataset(Dataset):
    def __init__(self, size):
        self.size = size

    def __len__(self):
        return self.size

    def __getitem__(self, idx):
        return torch.randn(3, 28, 28), torch.tensor(1)

# 定义优化器和损失函数
def train_model(model, device, dataloader, epochs):
    model.to(device)
    optimizer = optim.SGD(model.parameters(), lr=0.01)
    criterion = nn.CrossEntropyLoss()

    for epoch in range(epochs):
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
        print(f"Process {os.getpid()} - Epoch {epoch+1}, Loss: {loss.item()}")
    return loss.item()

# 创建数据加载器
dataset = SimpleDataset(1000)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True, num_workers=20)

# 设置环境变量，指定使用的GPU
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# 创建8个模型，都在gpu0
def main():
    start_time = time.time()
    processes = []
    devices = [torch.device("cuda:0")] * 8

    # 创建进程并开始训练
    for i, device in enumerate(devices):
        model = CNN()
        p = multiprocessing.Process(target=train_model, args=(model, device, dataloader, 5))
        processes.append(p)
        p.start()

    # 等待所有进程完成
    for p in processes:
        p.join()

    end_time = time.time()
    print(f"Total time for 8 models: {end_time - start_time} seconds")

if __name__ == "__main__":
    main()

## 8个进程，4090和3070各运行4个

耗时42.74378538131714秒

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import os
import multiprocessing
import time

# 定义一个简单的CNN模型
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv = nn.Conv2d(3, 16, 3, 1)
        self.fc = nn.Linear(16 * 26 * 26, 10)

    def forward(self, x):
        x = self.conv(x)
        x = torch.relu(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

# 定义数据集和数据加载器
class SimpleDataset(Dataset):
    def __init__(self, size):
        self.size = size

    def __len__(self):
        return self.size

    def __getitem__(self, idx):
        return torch.randn(3, 28, 28), torch.tensor(1)

# 定义优化器和损失函数
def train_model(model, device, dataloader, epochs):
    model.to(device)
    optimizer = optim.SGD(model.parameters(), lr=0.01)
    criterion = nn.CrossEntropyLoss()

    for epoch in range(epochs):
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
        print(f"Process {os.getpid()} - Epoch {epoch+1}, Loss: {loss.item()}")
    return loss.item()

# 创建数据加载器
dataset = SimpleDataset(1000)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True, num_workers=20)

# 设置环境变量，指定使用的GPU
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"

# 创建8个模型，4个在gpu0，4个在gpu1
def main():
    start_time = time.time()
    processes = []
    devices = [torch.device("cuda:0")] * 4 + [torch.device("cuda:1")] * 4

    # 创建进程并开始训练
    for i, device in enumerate(devices):
        model = CNN()
        p = multiprocessing.Process(target=train_model, args=(model, device, dataloader, 5))
        processes.append(p)
        p.start()

    # 等待所有进程完成
    for p in processes:
        p.join()

    end_time = time.time()
    print(f"Total time for 8 models: {end_time - start_time} seconds")

if __name__ == "__main__":
    main()

## 4个进程，每个进程都跨GPU运算

There is an imbalance between your GPUs. You may want to exclude GPU 1 which has less than 75% of the memory or cores of GPU 0. You can do so by setting the device_ids argument to DataParallel, or by setting the CUDA_VISIBLE_DEVICES environment variable.

warnings.warn(imbalance_warn.format(device_ids[min_pos], device_ids[max_pos]))

耗时49.47641134262085秒

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import os
import multiprocessing
import time

# 定义一个简单的CNN模型
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv = nn.Conv2d(3, 16, 3, 1)
        self.fc = nn.Linear(16 * 26 * 26, 10)

    def forward(self, x):
        x = self.conv(x)
        x = torch.relu(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

# 定义数据集和数据加载器
class SimpleDataset(Dataset):
    def __init__(self, size):
        self.size = size

    def __len__(self):
        return self.size

    def __getitem__(self, idx):
        return torch.randn(3, 28, 28), torch.tensor(1)

# 定义优化器和损失函数，并训练模型
def train_model(dataloader, epochs):
    # 创建模型
    model = CNN()

    # 定义设备
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    # 检查GPU数量并设置DataParallel
    if torch.cuda.device_count() > 1:
        print(f"Using {torch.cuda.device_count()} GPUs")
        model = nn.DataParallel(model)

    optimizer = optim.SGD(model.parameters(), lr=0.01)
    criterion = nn.CrossEntropyLoss()

    for epoch in range(epochs):
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
        print(f"Process {os.getpid()} - Epoch {epoch+1}, Loss: {loss.item()}")
    return loss.item()

# 创建数据加载器
dataset = SimpleDataset(1000)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True, num_workers=20)

# 设置环境变量，指定使用的GPU
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"

# 创建4个进程，每个进程都在两个GPU上运行
def main():
    start_time = time.time()
    processes = []

    # 创建进程并开始训练
    for _ in range(4):
        p = multiprocessing.Process(target=train_model, args=(dataloader, 5))
        processes.append(p)
        p.start()

    # 等待所有进程完成
    for p in processes:
        p.join()

    end_time = time.time()
    print(f"Total time for 8 models: {end_time - start_time} seconds")

if __name__ == "__main__":
    main()

## 单进程处理1小时视频

Total time for loading video: 162.94 seconds

Total time for training model: 18.84 seconds

Total time for the whole process 181.79 seconds

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import cv2
import numpy as np
import time

class SimpleVideoCNN(nn.Module):
    def __init__(self, input_shape):
        super(SimpleVideoCNN, self).__init__()

        self.conv3d_1 = nn.Conv3d(3, 4, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=1)
        self.pool_1 = nn.MaxPool3d(kernel_size=(2, 2, 2))

        self.conv3d_2 = nn.Conv3d(4, 8, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=1)
        self.pool_2 = nn.MaxPool3d(kernel_size=(2, 2, 2))

        # 动态计算全连接层输入大小
        with torch.no_grad():
            dummy_input = torch.zeros(input_shape)
            output = self.pool_2(self.conv3d_2(self.pool_1(self.conv3d_1(dummy_input))))
            self.flatten_size = output.numel() // dummy_input.size(0)

        self.fc1 = nn.Linear(self.flatten_size, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = torch.relu(self.conv3d_1(x))
        x = self.pool_1(x)
        x = torch.relu(self.conv3d_2(x))
        x = self.pool_2(x)
        x = x.view(x.size(0), -1)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x


# 读取本地视频并预处理
def load_video(video_path, fps_to_process=None, frame_size=(64, 64)):
    cap = cv2.VideoCapture(video_path)
    frames = []

    # 获取视频的帧率
    original_fps = cap.get(cv2.CAP_PROP_FPS)
    frame_interval = int(original_fps // fps_to_process) if fps_to_process and fps_to_process < original_fps else 1

    frame_idx = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:  # 没有读取到帧时退出循环
            break
        if frame_idx % frame_interval == 0:
            frame = cv2.resize(frame, frame_size)  # 缩放帧大小
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # 转换为 RGB 格式
            frames.append(frame)
        frame_idx += 1

    cap.release()

    if len(frames) == 0:
        raise ValueError(f"No frames could be read from the video at {video_path}.")

    # 转换为 Tensor 格式，形状为 (frames, height, width, channels)
    video_tensor = torch.tensor(np.array(frames), dtype=torch.float32).permute(3, 0, 1, 2) / 255.0
    return video_tensor.unsqueeze(0)  # 增加 batch 维度

# 训练函数
def train_model(model, device, video_tensor, epochs):
    model.to(device)
    optimizer = optim.SGD(model.parameters(), lr=0.01)
    criterion = nn.CrossEntropyLoss()

    # 模拟标签
    label = torch.tensor([1], device=device)

    for epoch in range(epochs):
        model.train()
        video_tensor = video_tensor.to(device)
        optimizer.zero_grad()
        output = model(video_tensor)
        loss = criterion(output, label)
        loss.backward()
        optimizer.step()
        print(f"Epoch {epoch+1}, Loss: {loss.item()}")

# 主函数
def main():
    total_start_time = time.time()
    video_path = "/data/dengqi_code/Video_Test.mp4"
    fps_to_process = 2  # 每秒处理的帧数
    frame_size = (64, 64)  # 降低分辨率以节省内存
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    print('Loading video...')
    load_video_start_time = time.time()
    video_tensor = load_video(video_path, fps_to_process=fps_to_process, frame_size=frame_size)
    load_video_end_time = time.time()

    print('Training model...')
    train_model_start_time = time.time()
    input_shape = (1, *video_tensor.shape[1:])  
    model = SimpleVideoCNN(input_shape)
    train_model(model, device, video_tensor, epochs=5)
    train_model_end_time = time.time()
    total_end_time = time.time()

    print(f"Total time for loading video: {load_video_end_time - load_video_start_time:.2f} seconds")
    print(f"Total time for training model: {train_model_end_time - train_model_start_time:.2f} seconds")
    print(f"Total time for the whole process {total_end_time - total_start_time:.2f} seconds")

if __name__ == "__main__":
    main()

Using device: cuda:0
Loading video...
Training model...
Epoch 1, Loss: 2.241482734680176
Epoch 2, Loss: 0.0029186292085796595
Epoch 3, Loss: 0.002448895713314414
Epoch 4, Loss: 0.0021068297792226076
Epoch 5, Loss: 0.0018468719208613038
Total time for loading video: 162.94 seconds
Total time for training model: 18.84 seconds
Total time for the whole process 181.79 seconds


## 分割视频，单GPU（4090）多进程共同处理1小时视频

num_segments = 16，total_time = 85.60

num_segments = 8，total_time = 67.5

num_segments = 4，total_time = 59.13

num_segments = 2，total_time = 119.64

num_segments = 1，total_time = 189.64

需要注意，进程越多，CPU负载越大，当CPU、内存的负载超过达到100%，整体运行效率越低。当前算力下，16进程时，CPU负载已接近100%

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import cv2
import numpy as np
import multiprocessing
import os
import time


class SimpleVideoCNN(nn.Module):
    def __init__(self, input_shape):
        super(SimpleVideoCNN, self).__init__()
        self.conv3d_1 = nn.Conv3d(3, 4, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=1)
        self.pool_1 = nn.MaxPool3d(kernel_size=(2, 2, 2))
        self.conv3d_2 = nn.Conv3d(4, 8, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=1)
        self.pool_2 = nn.MaxPool3d(kernel_size=(2, 2, 2))

        with torch.no_grad():
            dummy_input = torch.zeros(input_shape)
            output = self.pool_2(self.conv3d_2(self.pool_1(self.conv3d_1(dummy_input))))
            self.flatten_size = output.numel() // dummy_input.size(0)

        self.fc1 = nn.Linear(self.flatten_size, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = torch.relu(self.conv3d_1(x))
        x = self.pool_1(x)
        x = torch.relu(self.conv3d_2(x))
        x = self.pool_2(x)
        x = x.view(x.size(0), -1)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x


def load_video(video_path, start_frame, end_frame, fps_to_process, frame_size=(64, 64)):
    cap = cv2.VideoCapture(video_path)
    original_fps = cap.get(cv2.CAP_PROP_FPS)
    frame_interval = int(original_fps // fps_to_process) if fps_to_process and fps_to_process < original_fps else 1
    frames = []

    cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)

    frame_idx = 0
    while cap.isOpened():
        current_frame = int(cap.get(cv2.CAP_PROP_POS_FRAMES))
        if current_frame >= end_frame:
            break

        ret, frame = cap.read()
        if not ret:
            break

        if frame_idx % frame_interval == 0:
            frame = cv2.resize(frame, frame_size)
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frames.append(frame)

        frame_idx += 1

    cap.release()

    if len(frames) == 0:
        raise ValueError(f"No frames could be read between {start_frame} and {end_frame}.")

    video_tensor = torch.tensor(np.array(frames), dtype=torch.float32).permute(3, 0, 1, 2) / 255.0
    return video_tensor.unsqueeze(0)


def train_model(model, device, video_tensor, epochs):
    model.to(device)
    optimizer = optim.SGD(model.parameters(), lr=0.01)
    criterion = nn.CrossEntropyLoss()

    label = torch.tensor([1], device=device)

    for epoch in range(epochs):
        model.train()
        video_tensor = video_tensor.to(device)
        optimizer.zero_grad()
        output = model(video_tensor)
        loss = criterion(output, label)
        loss.backward()
        optimizer.step()
        print(f"Process {os.getpid()} - Epoch {epoch+1}, Loss: {loss.item()}")


def process_video_segment(video_path, start_frame, end_frame, fps_to_process, device, frame_size=(64, 64), epochs=5):
    print(f"Process {os.getpid()} handling frames {start_frame} to {end_frame} with fps_to_process={fps_to_process} on {device}")

    # Load video segment
    load_video_start_time = time.time()
    video_tensor = load_video(video_path, start_frame, end_frame, fps_to_process, frame_size)
    load_video_end_time = time.time()

    # Initialize and train model
    input_shape = (1, *video_tensor.shape[1:])
    model = SimpleVideoCNN(input_shape)
    train_model_start_time = time.time()
    train_model(model, device, video_tensor, epochs)
    train_model_end_time = time.time()

    # Output timing details
    print(f"Process {os.getpid()} - Time for loading video: {load_video_end_time - load_video_start_time:.2f} seconds")
    print(f"Process {os.getpid()} - Time for training model: {train_model_end_time - train_model_start_time:.2f} seconds")


def main():
    total_start_time = time.time()

    video_path = "/data/dengqi_code/Video_Test.mp4"
    total_frames = int(cv2.VideoCapture(video_path).get(cv2.CAP_PROP_FRAME_COUNT))
    original_fps = cv2.VideoCapture(video_path).get(cv2.CAP_PROP_FPS)
    num_segments = 2  # Number of segments to split video into
    segment_length = total_frames // num_segments
    fps_to_process = 2  # Target frames per second for processing
    frame_size = (64, 64)
    epochs = 5

    device = torch.device("cuda:0")  # Use cuda:0 for all processes
    processes = []

    for i in range(num_segments):
        start_frame = i * segment_length
        end_frame = total_frames if i == num_segments - 1 else (i + 1) * segment_length
        p = multiprocessing.Process(target=process_video_segment,
                                    args=(video_path, start_frame, end_frame, fps_to_process, device, frame_size, epochs))
        processes.append(p)
        p.start()

    for p in processes:
        p.join()

    total_end_time = time.time()
    print(f"Total time for the whole process: {total_end_time - total_start_time:.2f} seconds")


if __name__ == "__main__":
    main()

Process 1023530 handling frames 0 to 43218 with fps_to_process=2 on cuda:0
Process 1023533 handling frames 43218 to 86436 with fps_to_process=2 on cuda:0
Process 1023530 - Epoch 1, Loss: 2.2617197036743164
Process 1023530 - Epoch 2, Loss: 0.06742236763238907
Process 1023530 - Epoch 3, Loss: 2.7894584491150454e-05
Process 1023530 - Epoch 4, Loss: 2.777537883957848e-05
Process 1023530 - Epoch 5, Loss: 2.7656173188006505e-05
Process 1023530 - Time for loading video: 90.13 seconds
Process 1023530 - Time for training model: 11.12 seconds
Process 1023533 - Epoch 1, Loss: 2.262174606323242
Process 1023533 - Epoch 2, Loss: 0.06669007241725922
Process 1023533 - Epoch 3, Loss: 3.4689302992774174e-05
Process 1023533 - Epoch 4, Loss: 3.421248038648628e-05
Process 1023533 - Epoch 5, Loss: 3.397406908334233e-05
Process 1023533 - Time for loading video: 106.65 seconds
Process 1023533 - Time for training model: 9.51 seconds
Total time for the whole process: 119.64 seconds


## 分割视频，多GPU（3070+4090）多进程共同处理1小时视频

num_segments = 8，total_time = 32.65

num_segments = 4，total_time = 45.27

num_segments = 2，total_time = 72.45

注：进程总数为16时，3070显存过小，无法单独运行8个进程。在总进程数为8时，CPU负载已接近100%

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import cv2
import numpy as np
import multiprocessing
import os
import time


class SimpleVideoCNN(nn.Module):
    def __init__(self, input_shape):
        super(SimpleVideoCNN, self).__init__()
        self.conv3d_1 = nn.Conv3d(3, 4, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=1)
        self.pool_1 = nn.MaxPool3d(kernel_size=(2, 2, 2))
        self.conv3d_2 = nn.Conv3d(4, 8, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=1)
        self.pool_2 = nn.MaxPool3d(kernel_size=(2, 2, 2))

        with torch.no_grad():
            dummy_input = torch.zeros(input_shape)
            output = self.pool_2(self.conv3d_2(self.pool_1(self.conv3d_1(dummy_input))))
            self.flatten_size = output.numel() // dummy_input.size(0)

        self.fc1 = nn.Linear(self.flatten_size, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = torch.relu(self.conv3d_1(x))
        x = self.pool_1(x)
        x = torch.relu(self.conv3d_2(x))
        x = self.pool_2(x)
        x = x.view(x.size(0), -1)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x


def load_video(video_path, start_frame, end_frame, fps_to_process, frame_size=(64, 64)):
    cap = cv2.VideoCapture(video_path)
    original_fps = cap.get(cv2.CAP_PROP_FPS)
    frame_interval = int(original_fps // fps_to_process) if fps_to_process and fps_to_process < original_fps else 1
    frames = []

    cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)

    frame_idx = 0
    while cap.isOpened():
        current_frame = int(cap.get(cv2.CAP_PROP_POS_FRAMES))
        if current_frame >= end_frame:
            break

        ret, frame = cap.read()
        if not ret:
            break

        if frame_idx % frame_interval == 0:
            frame = cv2.resize(frame, frame_size)
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frames.append(frame)

        frame_idx += 1

    cap.release()

    if len(frames) == 0:
        raise ValueError(f"No frames could be read between {start_frame} and {end_frame}.")

    video_tensor = torch.tensor(np.array(frames), dtype=torch.float32).permute(3, 0, 1, 2) / 255.0
    return video_tensor.unsqueeze(0)


def train_model(model, device, video_tensor, epochs):
    model.to(device)
    optimizer = optim.SGD(model.parameters(), lr=0.01)
    criterion = nn.CrossEntropyLoss()

    label = torch.tensor([1], device=device)

    for epoch in range(epochs):
        model.train()
        video_tensor = video_tensor.to(device)
        optimizer.zero_grad()
        output = model(video_tensor)
        loss = criterion(output, label)
        loss.backward()
        optimizer.step()
        print(f"Process {os.getpid()} - Epoch {epoch+1}, Loss: {loss.item()}")


def process_video_segment(video_path, start_frame, end_frame, fps_to_process, device, frame_size=(64, 64), epochs=5):
    print(f"Process {os.getpid()} handling frames {start_frame} to {end_frame} with fps_to_process={fps_to_process} on {device}")

    # Load video segment
    load_video_start_time = time.time()
    video_tensor = load_video(video_path, start_frame, end_frame, fps_to_process, frame_size)
    load_video_end_time = time.time()

    # Initialize and train model
    input_shape = (1, *video_tensor.shape[1:])
    model = SimpleVideoCNN(input_shape)
    train_model_start_time = time.time()
    train_model(model, device, video_tensor, epochs)
    train_model_end_time = time.time()

    # Output timing details
    print(f"Process {os.getpid()} - Time for loading video: {load_video_end_time - load_video_start_time:.2f} seconds")
    print(f"Process {os.getpid()} - Time for training model: {train_model_end_time - train_model_start_time:.2f} seconds")


def main():
    total_start_time = time.time()

    video_path = "/data/dengqi_code/Video_Test.mp4"
    total_frames = int(cv2.VideoCapture(video_path).get(cv2.CAP_PROP_FRAME_COUNT))
    original_fps = cv2.VideoCapture(video_path).get(cv2.CAP_PROP_FPS)
    num_segments = 16  # Number of segments to split video into
    segment_length = total_frames // num_segments
    fps_to_process = 2  # Target frames per second for processing
    frame_size = (64, 64)
    epochs = 5

    # Assign devices to processes
    devices = [torch.device("cuda:0")] * (num_segments // 2) + [torch.device("cuda:1")] * (num_segments // 2)
    processes = []

    # Launch processes
    for i, device in enumerate(devices):
        start_frame = i * segment_length
        end_frame = total_frames if i == num_segments - 1 else (i + 1) * segment_length
        p = multiprocessing.Process(target=process_video_segment,
                                    args=(video_path, start_frame, end_frame, fps_to_process, device, frame_size, epochs))
        processes.append(p)
        p.start()

    # Wait for all processes to finish
    for p in processes:
        p.join()

    total_end_time = time.time()
    print(f"Total time for the whole process: {total_end_time - total_start_time:.2f} seconds")


if __name__ == "__main__":
    main()

Process 1048222 handling frames 0 to 5402 with fps_to_process=2 on cuda:0
Process 1048225 handling frames 5402 to 10804 with fps_to_process=2 on cuda:0
Process 1048230 handling frames 10804 to 16206 with fps_to_process=2 on cuda:0
Process 1048235 handling frames 16206 to 21608 with fps_to_process=2 on cuda:0
Process 1048240 handling frames 21608 to 27010 with fps_to_process=2 on cuda:0
Process 1048261 handling frames 27010 to 32412 with fps_to_process=2 on cuda:0
Process 1048282 handling frames 32412 to 37814 with fps_to_process=2 on cuda:0
Process 1048313 handling frames 37814 to 43216 with fps_to_process=2 on cuda:0
Process 1048340 handling frames 43216 to 48618 with fps_to_process=2 on cuda:1
Process 1048350 handling frames 48618 to 54020 with fps_to_process=2 on cuda:1
Process 1048368 handling frames 54020 to 59422 with fps_to_process=2 on cuda:1
Process 1048405 handling frames 59422 to 64824 with fps_to_process=2 on cuda:1
Process 1048438 handling frames 64824 to 70226 with fps_to

Process Process-16:
Traceback (most recent call last):
  File "/data/miniconda3/envs/ATSC/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
Process Process-11:
  File "/data/miniconda3/envs/ATSC/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
Process Process-9:
Process Process-10:
Process Process-14:
  File "/tmp/ipykernel_1048154/3020333015.py", line 102, in process_video_segment
    train_model(model, device, video_tensor, epochs)
Traceback (most recent call last):
Process Process-12:
Process Process-13:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/tmp/ipykernel_1048154/3020333015.py", line 83, in train_model
    output = model(video_tensor)
Process Process-15:
  File "/data/miniconda3/envs/ATSC/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
  File "/data/min

Process 1048261 - Epoch 1, Loss: 2.3601021766662598
Process 1048261 - Epoch 2, Loss: 1.724611520767212
Process 1048261 - Epoch 3, Loss: 0.07725840061903
Process 1048261 - Epoch 4, Loss: 0.007226163987070322
Process 1048261 - Epoch 5, Loss: 0.004976979922503233
Process 1048261 - Time for loading video: 13.05 seconds
Process 1048261 - Time for training model: 17.61 seconds
Process 1048235 - Epoch 1, Loss: 2.3586387634277344
Process 1048235 - Epoch 2, Loss: 1.7322229146957397
Process 1048235 - Epoch 3, Loss: 0.08189123868942261
Process 1048235 - Epoch 4, Loss: 0.006522556766867638
Process 1048235 - Epoch 5, Loss: 0.00460789306089282
Process 1048235 - Time for loading video: 13.86 seconds
Process 1048235 - Time for training model: 17.88 seconds
Process 1048225 - Epoch 1, Loss: 2.359348773956299
Process 1048225 - Epoch 2, Loss: 1.724930763244629
Process 1048225 - Epoch 3, Loss: 0.07835948467254639
Process 1048225 - Epoch 4, Loss: 0.007175271399319172
Process 1048225 - Epoch 5, Loss: 0.00495